In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 15.6 MB/s 
     |████████████████████████████████| 1.3 MB 72.7 MB/s 
     |████████████████████████████████| 163 kB 70.2 MB/s 
     |████████████████████████████████| 7.6 MB 52.2 MB/s 


In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 13.7 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 209 kB 86.5 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 147 kB 74.9 MB/s 
     |████████████████████████████████| 112 kB 70.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=a20cf3c764415646ee30c9d5eca363dfc54ae0b8309fc94861eb137d67cd6f5a
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
from transformers import AutoModel, AutoConfig, AutoTokenizer, get_cosine_schedule_with_warmup, get_polynomial_decay_schedule_with_warmup, get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup
import pandas as pd
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import mean_squared_error
import random
import time
from torch.utils import checkpoint
import math
import gc
from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs
import warnings
import torch.nn.functional as F
import optuna

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [2]:
#######
#### BEST PARAMS FOR THE DEBERTA V3 LARGE MODEL
# {'learning_rate': 1.7027701828938127e-05,
#  'layer_wise_learning_rate_decay': 0.8780935762069765,
#  'learning_rate_schduler': 'polynomial',
#  'reinit_layers': 1}
#######


#####
#### BEST PARAMS FOR THE DEBERTA V3 BASE MODEL
# {'learning_rate': 0.0002634969863920811, 
#  'layer_wise_learning_rate_decay': 0.7867664854455205, 
#  'learning_rate_schduler': 'polynomial', 
#  'reinit_layers': 3}
#####

In [3]:
import transformers
transformers.logging.set_verbosity_error()

In [4]:
warnings.simplefilter('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [7]:
class CFG:
    train_file = "/content/drive/MyDrive/Kaggle/FeedbackPrize3/train_folds.csv"
    fold = 0
    batch_size = 8
    num_workers = 4
    target_columns = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    hidden_dropout_prob = 0.0
    reinit_weights = False
    reinit_layers = 1
    lr = 2e-5
    llrd = 0.9
    warmup_ratio = 0.0
    use_awp = False
    adv_lr = 0.0002
    adv_eps = 0.001
    model_name = "microsoft/deberta-v3-large"
    gradient_accumulation_steps = 2
    max_grad_norm = 10
    print_freq = 20
    epochs = 3
    specific_max_len = 512
    token_dropout = False
    token_dropout_prob = 0.1
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    gradient_checkpointing_enable = True
    save_dir = "deberta-v3-large"
    save_model_name = "deberta-v3-large"

In [8]:
#Preprocessing Functions

def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

In [9]:
#Utiliy functions 
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))
        
def MCRMSE(y_trues, y_preds):
    ## Mapping real numbers to classes
    classes = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
    class_mapping = dict()
    for i, class_ in enumerate(classes):
      class_mapping[class_] = i
    
    inv_class_mapping = {v : k for k, v in class_mapping.items()}
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = np.vectorize(inv_class_mapping.get)(y_trues[:,i])
        y_pred = np.vectorize(inv_class_mapping.get)(y_preds[:,i])
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores

In [10]:
class Collate:
    def __init__(self, tokenizer, max_length = CFG.specific_max_len):
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch):
        
        batch_len = max([len(sample["ids"]) for sample in batch])
        
        output = dict()
        output["ids"] = [sample["ids"] for sample in batch]
        output["mask"] = [sample["mask"] for sample in batch]
        output["targets"] = [sample["targets"] for sample in batch]
        
        if self.tokenizer.padding_side == "right":
            output["ids"] = [s + [self.tokenizer.pad_token_id] * (batch_len - len(s)) for s in output["ids"]]
            output["mask"] = [s + [0] * (batch_len - len(s)) for s in output["mask"]]
        else:
            output["ids"] = [[self.tokenizer.pad_token_id] * (batch_len - len(s)) + s for s in output["ids"]]
            output["mask"] = [[0] * (batch_len - len(s)) + s for s in output["mask"]]
            
            
        output["ids"] = torch.tensor(output["ids"], dtype = torch.long)
        output["mask"] = torch.tensor(output["mask"], dtype = torch.long)
        output["targets"] = torch.tensor(output["targets"], dtype = torch.long)
        
        return output

In [11]:
class Dataset:
    def __init__(self, texts, targets, tokenizer, is_train = True):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.is_train = is_train
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        
        text = self.texts[idx]
        targets = self.targets[idx]
        
        
        if CFG.specific_max_len is not None:
          encoding = self.tokenizer(text, add_special_tokens = True, max_length = CFG.specific_max_len, padding = False, truncation = 'longest_first')
        else:
          encoding = self.tokenizer(text, add_special_tokens = True)
        
        sample = dict()

        if CFG.token_dropout and self.is_train:
          idxs = np.random.choice(np.arange(1, len(encoding["input_ids"]) - 1), size = int(CFG.token_dropout_prob * len(encoding["input_ids"])), replace = False)
          ids = np.array(encoding["input_ids"])
          ids[idxs] = self.tokenizer.mask_token_id
          encoding["input_ids"] = ids.tolist()
          
        
        sample["ids"] = encoding["input_ids"]
        sample["mask"] = encoding["attention_mask"]  
        sample["targets"] = targets
        
        return sample

In [12]:
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

class SmoothRMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.SmoothL1Loss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

class SmoothRMSEComp(nn.Module):
  def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.SmoothL1Loss(reduction='none')
        self.reduction = reduction
        self.eps = eps

  def forward(self, y_pred, y_true):
    loss = torch.sqrt(self.mse(y_pred, y_true).mean(dim = 0)).mean(dim = 0)
    return loss  


class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, features):

        all_layer_embedding = torch.stack(features)
        all_layer_embedding = all_layer_embedding[self.layer_start:, :, :, :]

        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()

        return weighted_average

In [13]:
#AWP
class AWP:
    def __init__(
        self,
        model,
        optimizer,
        adv_param="weight",
        adv_lr=1,
        adv_eps=0.2,
        start_epoch=0,
        adv_step=1,
        scaler=None
    ):
        self.model = model
        self.optimizer = optimizer
        self.adv_param = adv_param
        self.adv_lr = adv_lr
        self.adv_eps = adv_eps
        self.start_epoch = start_epoch
        self.adv_step = adv_step
        self.backup = {}
        self.backup_eps = {}
        self.scaler = scaler

    def attack_backward(self, x, y, attention_mask,epoch):
        if (self.adv_lr == 0) or (epoch < self.start_epoch):
            return None

        self._save() 
        for i in range(self.adv_step):
            self._attack_step() 
            with torch.cuda.amp.autocast():
                adv_loss, tr_logits = self.model(ids=x, mask=attention_mask, targets=y)
                adv_loss = adv_loss.mean()
            self.optimizer.zero_grad()
            self.scaler.scale(adv_loss).backward()
            
        self._restore()

    def _attack_step(self):
        e = 1e-6
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                norm1 = torch.norm(param.grad)
                norm2 = torch.norm(param.data.detach())
                if norm1 != 0 and not torch.isnan(norm1):
                    r_at = self.adv_lr * param.grad / (norm1 + e) * (norm2 + e)
                    param.data.add_(r_at)
                    param.data = torch.min(
                        torch.max(param.data, self.backup_eps[name][0]), self.backup_eps[name][1]
                    )
                # param.data.clamp_(*self.backup_eps[name])

    def _save(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad and param.grad is not None and self.adv_param in name:
                if name not in self.backup:
                    self.backup[name] = param.data.clone()
                    grad_eps = self.adv_eps * param.abs().detach()
                    self.backup_eps[name] = (
                        self.backup[name] - grad_eps,
                        self.backup[name] + grad_eps,
                    )

    def _restore(self,):
        for name, param in self.model.named_parameters():
            if name in self.backup:
                param.data = self.backup[name]
        self.backup = {}
        self.backup_eps = {}

In [14]:
class Model(nn.Module):
    def __init__(self, model_name, params = None):
        super(Model, self).__init__()
        
        self.model_name = model_name

        hidden_dropout_prob: float = CFG.hidden_dropout_prob
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(model_name)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "attention_probs_dropout_prob" : hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
                "num_labels": 6,
            }
        )
        
        self.config = config
        self.params = params
        
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        if CFG.gradient_checkpointing_enable:
            self.transformer.gradient_checkpointing_enable()
        
        self.output1 = nn.Linear(config.hidden_size, 9)
        self.output2 = nn.Linear(config.hidden_size, 9)
        self.output3 = nn.Linear(config.hidden_size, 9)
        self.output4 = nn.Linear(config.hidden_size, 9)
        self.output5 = nn.Linear(config.hidden_size, 9)
        self.output6 = nn.Linear(config.hidden_size, 9)
        self.loss = nn.CrossEntropyLoss()

        if CFG.reinit_weights:
          self.init_weights_(params["reinit_layers"])

    def init_weights_(self, reinit_layers):
      for layer in self.transformer.encoder.layer[-reinit_layers:]:
        for module in layer.modules():
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.Embedding):
                module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                if module.padding_idx is not None:
                    module.weight.data[module.padding_idx].zero_()
            elif isinstance(module, nn.LayerNorm):
                module.bias.data.zero_()
                module.weight.data.fill_(1.0)

    def get_grouped_llrd_optimizer_scheduler(self, num_train_steps, params):
      no_decay = ["bias", "LayerNorm.weight"]
      top_params = list(self.output.named_parameters())
      # initialize lr for task specific layer
      optimizer_grouped_parameters = [
              {
                  "params": [p for n, p in top_params if not any(nd in n for nd in no_decay)],
                  "weight_decay": 0.001,
                  "lr": params["lr"],
              },
              {
                  "params": [p for n, p in top_params if any(nd in n for nd in no_decay)],
                  "weight_decay": 0.0,
                  "lr": params["lr"],
              },
          ]

      bottom_params = [(n,p) for n,p in self.named_parameters() if n in ["transformer.encoder.rel_embeddings.weight","transformer.encoder.LayerNorm.weight","transformer.encoder.LayerNorm.bias"]]

      optimizer_grouped_parameters += [
              {
                  "params": [p for n, p in bottom_params if not any(nd in n for nd in no_decay)],
                  "weight_decay": 0.001,
                  "lr": params["lr"],
              },
              {
                  "params": [p for n, p in bottom_params if any(nd in n for nd in no_decay)],
                  "weight_decay": 0.0,
                  "lr": params["lr"],
              },
          ]
      # initialize lrs for every layer
      num_layers = self.config.num_hidden_layers
      layers = [self.transformer.embeddings] + list(self.transformer.encoder.layer)
      layers.reverse()
      lr = params["lr"]
      for i,layer in enumerate(layers):
        lr *= params["llrd"]

        optimizer_grouped_parameters += [
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
                "lr": lr,
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": lr,
            },
        ]
      opt = torch.optim.AdamW(optimizer_grouped_parameters)
      if params["scheduler"] == "polynomial":
        sch = get_polynomial_decay_schedule_with_warmup(
            opt,
            num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
            num_training_steps=num_train_steps,
            last_epoch=-1,
        )
      elif params["scheduler"] == "linear":
        sch = get_linear_schedule_with_warmup(
            opt,
            num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
            num_training_steps=num_train_steps,
            last_epoch=-1,
        )
      else:
        sch = get_cosine_schedule_with_warmup(
            opt,
            num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
            num_training_steps=num_train_steps,
            last_epoch=-1,
        )

      return opt, sch
        
    def get_optimizer_scheduler(self, num_train_steps):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        opt = torch.optim.AdamW(optimizer_parameters, lr=CFG.lr)
        sch = get_polynomial_decay_schedule_with_warmup(
            opt,
            num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
            num_training_steps=num_train_steps,
            last_epoch=-1,
        )
        return opt, sch

    def forward(self, ids, mask, token_type_ids=None, targets=None):
        if token_type_ids is not None:
            transformer_out = self.transformer( ids, mask, token_type_ids )
        else:
            transformer_out = self.transformer(ids, mask)
        sequence_output = transformer_out.last_hidden_state[:,0,:]
    
        logits1 = self.output1(sequence_output)
        logits2 = self.output2(sequence_output)
        logits3 = self.output3(sequence_output)
        logits4 = self.output4(sequence_output)
        logits5 = self.output5(sequence_output)
        logits6 = self.output6(sequence_output)

        
        loss1 = self.loss(logits1, targets[:,0])
        loss2 = self.loss(logits2, targets[:,1])
        loss3 = self.loss(logits3, targets[:,2])
        loss4 = self.loss(logits4, targets[:,3])
        loss5 = self.loss(logits5, targets[:,4])
        loss6 = self.loss(logits6, targets[:,5])

        loss = (loss1 + loss2 + loss3 + loss4 + loss5 + loss6)/ 6

        logits = torch.cat((logits1.unsqueeze(-1), logits2.unsqueeze(-1), logits3.unsqueeze(-1), logits4.unsqueeze(-1), logits5.unsqueeze(-1), logits6.unsqueeze(-1)), dim = -1)

        return loss, logits

In [16]:
def train(epoch, model, train_loader, valid_loader, optimizer, scheduler, device, awp, scaler, best_loss, fold):
    model.train()
    losses = AverageMeter()
    start = end = time.time()
    val_steps = len(train_loader) // 1
    for step, x in enumerate(train_loader):
        for k,v in x.items():
            x[k] = v.to(device)
        
        with autocast():    
            loss, logits = model(**x)
            
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        
        losses.update(loss.item() * CFG.gradient_accumulation_steps , CFG.batch_size)
        scaler.scale(loss).backward()
        if CFG.use_awp:
          awp.attack_backward(x["ids"],x["targets"],x["mask"],epoch) 
        
        
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.unscale_(optimizer)
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
        end = time.time()
        
        if ((step + 1) % CFG.print_freq == 0) or (step == (len(train_loader)-1)):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step + 1, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm= grad_norm,
                          lr=scheduler.get_lr()[0]))
        
        if ((step + 1) % val_steps == 0) or ((step + 1) == len(train_loader)):
          print("\nVALID LOOP\n")
          valid_loss = valid(epoch, model, valid_loader, device)
          print(f"\nThe valid loss for the current epoch is {valid_loss}\n")
          torch.cuda.empty_cache()
          gc.collect()
          if valid_loss < best_loss:
              best_loss = valid_loss
              if CFG.save_dir is not None:
                if not os.path.exists(CFG.save_dir):
                  os.mkdir(CFG.save_dir)
                save_path = os.path.join(CFG.save_dir, f"{CFG.save_model_name}_fold_{fold}.pth")
              else:
                save_path = f"{CFG.save_model_name}_fold_{fold}.pth"
              torch.save(model.state_dict(), save_path)
        model.train()
    return best_loss

def valid(epoch, model, valid_loader, device):
    model.eval()
    all_targets = []
    all_outputs = []
    losses = AverageMeter()
    with torch.no_grad():
        for step, x in enumerate(valid_loader):

            for k, v in x.items():
                x[k] = v.to(device)
            with autocast():
                loss, logits = model(**x)

            
            losses.update(loss.item(), CFG.batch_size)
            targets = x["targets"].cpu().numpy()
            outputs = torch.max(logits, dim = 1)[1].cpu().numpy()

            all_targets.append(targets)
            all_outputs.append(outputs)

            if ((step + 1) % CFG.print_freq == 0) or (step == (len(valid_loader)-1)):
                print('Epoch: [{0}][{1}/{2}] '
                      'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                      .format(epoch+1, step + 1, len(valid_loader), loss=losses))
        
    
    all_targets = np.vstack(all_targets)
    all_outputs = np.vstack(all_outputs)
    loss = get_score(all_targets, all_outputs)[0]
    
    del all_targets, all_outputs;
    return loss

In [17]:
def main(fold, params):

    ## Mapping real numbers to classes
    classes = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
    class_mapping = dict()
    for i, class_ in enumerate(classes):
      class_mapping[class_] = i

    
    torch.cuda.empty_cache()
    df = pd.read_csv(CFG.train_file)
    
    train_df = df.loc[df.kfold != fold]
    valid_df = df.loc[df.kfold == fold]
    
    train_texts = train_df["full_text"].apply(resolve_encodings_and_normalize).to_list()
    valid_texts = valid_df["full_text"].apply(resolve_encodings_and_normalize).to_list()
    
    train_targets = train_df[CFG.target_columns].applymap(lambda x : class_mapping[float(x)]).values.tolist()
    
    valid_targets = valid_df[CFG.target_columns].applymap(lambda x : class_mapping[float(x)]).values.tolist()
    
    train_ds = Dataset(train_texts, train_targets, CFG.tokenizer)
    valid_ds = Dataset(valid_texts, valid_targets, CFG.tokenizer, is_train = False)

    collate_fn = Collate(CFG.tokenizer)
    train_loader = torch.utils.data.DataLoader(train_ds, batch_size = CFG.batch_size, shuffle = True, collate_fn = collate_fn, num_workers = CFG.num_workers)
    valid_loader = torch.utils.data.DataLoader(valid_ds, batch_size = CFG.batch_size, shuffle = False, collate_fn = collate_fn, num_workers = CFG.num_workers)
    
    model = Model(CFG.model_name, params)

    ## Adding noisy tune -> NoisyTune: A Little Noise Can Help You Finetune Pretrained Language Models Better
    # noise_lambda = 0.15
    # sd = model.state_dict()
    # for name, param in model.named_parameters():
    #   sd[name][:] += (torch.rand(param.size()) - 0.5) * noise_lambda * torch.std(param)

    
    # model.load_state_dict(sd)

    
    num_train_steps = int(len(train_ds) / CFG.batch_size / CFG.gradient_accumulation_steps * CFG.epochs)
    optimizer, scheduler = model.get_optimizer_scheduler(num_train_steps)
    
    model = model.to(CFG.device)
    best_loss = np.inf
    scaler = GradScaler()
    if CFG.use_awp:
      print('Enable AWP')
      awp = AWP(model,
          optimizer,
          adv_lr=CFG.adv_lr,
          adv_eps=CFG.adv_eps,
          start_epoch=2,
          scaler=scaler)
    else:
      awp = None
    for epoch in range(CFG.epochs):
        print("\nTRAIN LOOP\n")
        best_loss = train(epoch, model, train_loader, valid_loader, optimizer, scheduler, CFG.device, awp, scaler, best_loss, fold)
        # print("\nVALID LOOP\n")
        # valid_loss = valid(epoch, model, valid_loader, CFG.device)
        
        # print(f"\nThe valid loss for the current epoch is {valid_loss}\n")
        # torch.cuda.empty_cache()
        # gc.collect()
        # if valid_loss < best_loss:
        #     best_loss = valid_loss
        #     if CFG.save_dir is not None:
        #       if not os.path.exists(CFG.save_dir):
        #         os.mkdir(CFG.save_dir)
        #       save_path = os.path.join(CFG.save_dir, f"{CFG.save_model_name}_fold_{fold}.pth")
        #     else:
        #       save_path = f"{CFG.save_model_name}_fold_{fold}.pth"
        #     torch.save(model.state_dict(), save_path)
            
    del model, optimizer, scheduler, train_loader, valid_loader, train_df, valid_df;
    gc.collect()

    return best_loss

In [18]:
for fold in range(5):
  print("-----"*30)
  print(f"\nRUNNING FOLD {fold}\n")
  print("-----"*30)
  params = {'lr': 2e-5,
            'llrd': 0.9,
            'scheduler': 'polynomial',
            'reinit_layers': 4}
  best_loss = main(fold, params)
  print("######")
  print(f"The best loss for fold {fold} is {best_loss}")
  print("######")
  gc.collect()

------------------------------------------------------------------------------------------------------------------------------------------------------

RUNNING FOLD 0

------------------------------------------------------------------------------------------------------------------------------------------------------

TRAIN LOOP

Epoch: [1][20/392] Elapsed 0m 35s (remain 10m 56s) Loss: 1.9588(2.1745) Grad: 6.7461  LR: 0.00001966  
Epoch: [1][40/392] Elapsed 1m 9s (remain 10m 13s) Loss: 1.6163(2.0151) Grad: 5.4931  LR: 0.00001932  
Epoch: [1][60/392] Elapsed 1m 44s (remain 9m 37s) Loss: 1.5365(1.9075) Grad: 4.7764  LR: 0.00001898  
Epoch: [1][80/392] Elapsed 2m 18s (remain 9m 1s) Loss: 1.4415(1.8316) Grad: 6.0434  LR: 0.00001864  
Epoch: [1][100/392] Elapsed 2m 53s (remain 8m 27s) Loss: 1.5514(1.7810) Grad: 5.2040  LR: 0.00001830  
Epoch: [1][120/392] Elapsed 3m 28s (remain 7m 53s) Loss: 1.3717(1.7397) Grad: 4.4591  LR: 0.00001796  
Epoch: [1][140/392] Elapsed 4m 4s (remain 7m 19s) Loss

KeyboardInterrupt: ignored

In [ ]:
#running hyperparameter tuning on first fold with optuna
def objective(trial):
  params = {
      "lr": trial.suggest_loguniform("learning_rate", 1e-7, 1e-4),
      "llrd" : trial.suggest_uniform("layer_wise_learning_rate_decay", 0.7, 1.0),
      "scheduler" : trial.suggest_categorical("learning_rate_schduler",["polynomial", "cosine","linear"]),
      "reinit_layers": trial.suggest_int("reinit_layers",0, 4)
  }
  best_loss= main(1,params)
  return best_loss


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2022-09-15 18:26:18,743] A new study created in memory with name: no-name-d9193ad8-4b94-4d31-a28c-de8c5b753dcc



TRAIN LOOP

Epoch: [1][20/196] Elapsed 1m 12s (remain 10m 33s) Loss: 1.3281(2.1507) Grad: 16.2040  LR: 0.00000612  
Epoch: [1][40/196] Elapsed 2m 23s (remain 9m 19s) Loss: 0.2657(1.2626) Grad: 3.9198  LR: 0.00001225  
Epoch: [1][60/196] Elapsed 3m 35s (remain 8m 7s) Loss: 0.1242(0.8913) Grad: 3.8266  LR: 0.00001837  
Epoch: [1][80/196] Elapsed 4m 46s (remain 6m 55s) Loss: 0.1186(0.6998) Grad: 4.7077  LR: 0.00002450  
Epoch: [1][100/196] Elapsed 5m 57s (remain 5m 43s) Loss: 0.0962(0.5867) Grad: 1.9993  LR: 0.00003062  
Epoch: [1][120/196] Elapsed 7m 9s (remain 4m 31s) Loss: 0.1393(0.5106) Grad: 2.8769  LR: 0.00003583  
Epoch: [1][140/196] Elapsed 8m 20s (remain 3m 20s) Loss: 0.1014(0.4560) Grad: 3.6234  LR: 0.00003562  
Epoch: [1][160/196] Elapsed 9m 32s (remain 2m 8s) Loss: 0.0952(0.4140) Grad: 2.8452  LR: 0.00003509  
Epoch: [1][180/196] Elapsed 10m 43s (remain 0m 57s) Loss: 0.1466(0.3822) Grad: 3.5936  LR: 0.00003426  
Epoch: [1][196/196] Elapsed 11m 39s (remain 0m 0s) Loss: 0.1468(

[I 2022-09-15 19:04:12,212] Trial 0 finished with value: 0.45831298828125 and parameters: {'learning_rate': 3.5828994394331454e-05, 'layer_wise_learning_rate_decay': 0.9767672591137755, 'learning_rate_schduler': 'cosine', 'reinit_layers': 1}. Best is trial 0 with value: 0.45831298828125.



TRAIN LOOP

Epoch: [1][20/196] Elapsed 1m 11s (remain 10m 32s) Loss: 0.6627(1.9961) Grad: 12.6625  LR: 0.00001022  
Epoch: [1][40/196] Elapsed 2m 23s (remain 9m 18s) Loss: 0.1617(1.1272) Grad: 5.9817  LR: 0.00002043  
Epoch: [1][60/196] Elapsed 3m 34s (remain 8m 6s) Loss: 0.1523(0.8059) Grad: 6.1189  LR: 0.00003065  
Epoch: [1][80/196] Elapsed 4m 46s (remain 6m 54s) Loss: 0.1614(0.6430) Grad: 4.9825  LR: 0.00004086  
Epoch: [1][100/196] Elapsed 5m 57s (remain 5m 43s) Loss: 0.0882(0.5389) Grad: 2.2667  LR: 0.00005108  
Epoch: [1][120/196] Elapsed 7m 9s (remain 4m 31s) Loss: 0.1492(0.4709) Grad: 5.0348  LR: 0.00005975  
Epoch: [1][140/196] Elapsed 8m 20s (remain 3m 20s) Loss: 0.0770(0.4227) Grad: 1.9766  LR: 0.00005941  
Epoch: [1][160/196] Elapsed 9m 32s (remain 2m 8s) Loss: 0.1981(0.3885) Grad: 4.4461  LR: 0.00005853  
Epoch: [1][180/196] Elapsed 10m 43s (remain 0m 57s) Loss: 0.1491(0.3605) Grad: 3.6420  LR: 0.00005714  
Epoch: [1][196/196] Elapsed 11m 39s (remain 0m 0s) Loss: 0.1232(

[I 2022-09-15 19:42:05,320] Trial 1 finished with value: 0.45570382475852966 and parameters: {'learning_rate': 5.975913614980313e-05, 'layer_wise_learning_rate_decay': 0.9419119594261866, 'learning_rate_schduler': 'cosine', 'reinit_layers': 1}. Best is trial 1 with value: 0.45570382475852966.


...

[I 2022-09-16 05:47:37,855] Trial 17 finished with value: 0.45891880989074707 and parameters: {'learning_rate': 7.707324684449108e-05, 'layer_wise_learning_rate_decay': 0.8339476407176097, 'learning_rate_schduler': 'polynomial', 'reinit_layers': 4}. Best is trial 14 with value: 0.4489534795284271.



TRAIN LOOP

Epoch: [1][20/196] Elapsed 1m 12s (remain 10m 36s) Loss: 2.6135(2.7590) Grad: 19.8303  LR: 0.00000063  
Epoch: [1][40/196] Elapsed 2m 23s (remain 9m 20s) Loss: 2.0224(2.5872) Grad: 19.9114  LR: 0.00000127  
Epoch: [1][60/196] Elapsed 3m 34s (remain 8m 7s) Loss: 1.1023(2.2502) Grad: 18.6274  LR: 0.00000190  
Epoch: [1][80/196] Elapsed 4m 46s (remain 6m 55s) Loss: 0.2280(1.8308) Grad: 6.7808  LR: 0.00000254  
Epoch: [1][100/196] Elapsed 5m 57s (remain 5m 43s) Loss: 0.2152(1.5102) Grad: 3.8052  LR: 0.00000317  
Epoch: [1][120/196] Elapsed 7m 9s (remain 4m 31s) Loss: 0.2582(1.2887) Grad: 2.1512  LR: 0.00000369  
Epoch: [1][140/196] Elapsed 8m 20s (remain 3m 20s) Loss: 0.1642(1.1284) Grad: 2.3194  LR: 0.00000353  
Epoch: [1][160/196] Elapsed 9m 32s (remain 2m 8s) Loss: 0.1995(1.0067) Grad: 3.5046  LR: 0.00000337  
Epoch: [1][180/196] Elapsed 10m 43s (remain 0m 57s) Loss: 0.1004(0.9093) Grad: 2.2256  LR: 0.00000321  
Epoch: [1][196/196] Elapsed 11m 38s (remain 0m 0s) Loss: 0.110

[I 2022-09-16 06:25:32,764] Trial 18 finished with value: 0.46429410576820374 and parameters: {'learning_rate': 3.712200194342814e-06, 'layer_wise_learning_rate_decay': 0.873797238234208, 'learning_rate_schduler': 'polynomial', 'reinit_layers': 2}. Best is trial 14 with value: 0.4489534795284271.



TRAIN LOOP

Epoch: [1][20/196] Elapsed 1m 12s (remain 10m 36s) Loss: 1.6407(2.3579) Grad: 19.6849  LR: 0.00000342  
Epoch: [1][40/196] Elapsed 2m 23s (remain 9m 21s) Loss: 0.1692(1.6092) Grad: 2.6283  LR: 0.00000683  
Epoch: [1][60/196] Elapsed 3m 35s (remain 8m 8s) Loss: 0.1660(1.1529) Grad: 1.6743  LR: 0.00001025  
Epoch: [1][80/196] Elapsed 4m 46s (remain 6m 56s) Loss: 0.1442(0.9106) Grad: 2.9029  LR: 0.00001366  
Epoch: [1][100/196] Elapsed 5m 58s (remain 5m 44s) Loss: 0.1284(0.7553) Grad: 5.7241  LR: 0.00001708  
Epoch: [1][120/196] Elapsed 7m 10s (remain 4m 32s) Loss: 0.0961(0.6498) Grad: 6.2203  LR: 0.00001985  
Epoch: [1][140/196] Elapsed 8m 21s (remain 3m 20s) Loss: 0.0909(0.5737) Grad: 2.3681  LR: 0.00001900  
Epoch: [1][160/196] Elapsed 9m 33s (remain 2m 8s) Loss: 0.0931(0.5165) Grad: 2.3086  LR: 0.00001815  
Epoch: [1][180/196] Elapsed 10m 44s (remain 0m 57s) Loss: 0.1277(0.4712) Grad: 2.2002  LR: 0.00001730  
Epoch: [1][196/196] Elapsed 11m 40s (remain 0m 0s) Loss: 0.1350

[I 2022-09-16 07:03:31,355] Trial 19 finished with value: 0.45386430621147156 and parameters: {'learning_rate': 1.9979077386798717e-05, 'layer_wise_learning_rate_decay': 0.8086618256858717, 'learning_rate_schduler': 'polynomial', 'reinit_layers': 2}. Best is trial 14 with value: 0.4489534795284271.


In [ ]:
study.best_trial.params

{'learning_rate': 1.7027701828938127e-05,
 'layer_wise_learning_rate_decay': 0.8780935762069765,
 'learning_rate_schduler': 'polynomial',
 'reinit_layers': 1}

In [ ]:
!cp -r deberta-v3-large/ /content/drive/MyDrive/Kaggle/FeedbackPrize3/